In [9]:
def model_1DM():
    model1dM = Sequential()
    model1dM.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
    model1dM.add(Conv1D(32, kernel_size=5))    # added new
    model1dM.add(MaxPooling1D(pool_size=(2)))
    model1dM.add(Conv1D(64, kernel_size=5))
    model1dM.add(Conv1D(64, kernel_size=5))    # added new
    model1dM.add(MaxPooling1D(pool_size=(2)))
    model1dM.add(Conv1D(128, kernel_size=3))
    model1dM.add(Conv1D(128, kernel_size=3))    # added new and changed to 128 from 64
    model1dM.add(MaxPooling1D(pool_size=(2)))
    model1dM.add(Dropout(0.25))
    model1dM.add(Flatten())
    model1dM.add(Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(l=0.1)))
    model1dM.add(Dropout(0.5))
    model1dM.add(Dense(1, activation='sigmoid'))
    model1dM.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model1dM
                     
def model_BNM():
                     
    modelBNM = Sequential()
    modelBNM.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
    modelBNM.add(MaxPooling1D(pool_size=(2)))
    modelBNM.add(Conv1D(32, kernel_size=5))
    modelBNM.add(MaxPooling1D(pool_size=(2)))
    modelBNM.add(Conv1D(64, kernel_size=3))
    modelBNM.add(MaxPooling1D(pool_size=(2)))
    modelBNM.add(Dropout(0.25))
    modelBNM.add(Flatten())
    # we can think of this chunk as the input layer
    modelBNM.add(Dense(128))
    modelBNM.add(BatchNormalization())
    modelBNM.add(Activation('relu'))
    modelBNM.add(Dropout(0.5))

    # we can think of this chunk as the hidden layer    
    modelBNM.add(Dense(64))
    modelBNM.add(BatchNormalization())
    modelBNM.add(Activation('relu'))
    modelBNM.add(Dropout(0.5))

    # we can think of this chunk as the output layer
    modelBNM.add(Dense(1))
    modelBNM.add(BatchNormalization())
    modelBNM.add(Activation('sigmoid'))

    # setting up the optimization of our weights 
    #sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    modelBNM.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return modelBNM
# running the fitting
                     
def BN_Regularization():
                    
# instantiate model
    modelBNrm = Sequential()
    modelBNrm.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
    modelBNrm.add(MaxPooling1D(pool_size=(2)))
    modelBNrm.add(Conv1D(32, kernel_size=5))
    modelBNrm.add(MaxPooling1D(pool_size=(2)))
    modelBNrm.add(Conv1D(64, kernel_size=3))
    modelBNrm.add(MaxPooling1D(pool_size=(2)))
    modelBNrm.add(Dropout(0.25))
    modelBNrm.add(Flatten())
    # we can think of this chunk as the input layer
    modelBNrm.add(Dense(128,kernel_regularizer=keras.regularizers.l2(l=0.1)))
    modelBNrm.add(BatchNormalization())
    modelBNrm.add(Activation('relu'))
    modelBNrm.add(Dropout(0.5))

    # we can think of this chunk as the hidden layer    
    modelBNrm.add(Dense(64,kernel_regularizer=keras.regularizers.l2(l=0.1)))
    modelBNrm.add(BatchNormalization())
    modelBNrm.add(Activation('relu'))
    modelBNrm.add(Dropout(0.5))

    # we can think of this chunk as the output layer
    modelBNrm.add(Dense(1,kernel_regularizer=keras.regularizers.l2(l=0.1)))
    modelBNrm.add(BatchNormalization())
    modelBNrm.add(Activation('sigmoid'))

    # setting up the optimization of our weights 
    #sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    modelBNrm.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model
    # running the fitting


In [10]:
# Making Wrapper
import tensorflow as tf
from sklearn.ensemble import BaggingClassifier

Model1DM = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_1DM,
                           epochs=50, 
                            verbose=False)

Model_BNM = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_BNM,
                           epochs=50,
                            verbose=False)

Model_BN_Regularization = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            BN_Regularization,
                           epochs=50,
                            verbose=False)


In [11]:
Model1DM._estimator_type = "classifier"
Model_BNM._estimator_type = "classifier"
Model_BN_Regularization._estimator_type = "classifier"


 

In [12]:

from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(
             estimators=[('model1DM',Model1DM),
                         ('Model_BNM', Model_BNM), 
                         ('Model_BN_Regularization', Model_BN_Regularization)], 
    
             voting='soft',
             flatten_transform=True)

In [13]:
# Ensemble learning training
for clf in (Model1DM, Model_BNM,Model_BN_Regularization, voting):
    clf.fit(final_a_ary, y_train)

NameError: name 'X_train' is not defined

In [15]:
#y_pred = clf.predict(final_b_bry)


In [17]:
#Printing accuracy score

#print(clf.__class__.__name__, accuracy_score(final_b_ary, y_pred))
#y_pred = clf.predict(X_test)